# Telecom Customer Data Preprocessing
## AI Customer Upsell Prediction System - Smart Segmentation Approach

This notebook handles:
- Data loading and cleaning
- Duplicate removal
- Missing value treatment
- Feature engineering (50+ advanced features)
- **SMART CUSTOMER SEGMENTATION** for targeted upselling
- **SATISFACTION-AWARE PRIORITIZATION**
- Data quality assessment

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')
sns.set_palette('viridis')

In [2]:
# Load the telecom dataset
print("📊 Loading Telecom Customer Dataset...")
# Update path to match your directory structure
df_raw = pd.read_csv('../../data/raw/telecom_data.csv')

print(f"Dataset Shape: {df_raw.shape}")
print(f"Columns: {list(df_raw.columns)}")
print("\n📋 First 5 rows:")
df_raw.head()

📊 Loading Telecom Customer Dataset...
Dataset Shape: (101174, 17)
Columns: ['Phone Number', 'Account Length', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn']

📋 First 5 rows:


,Phone Number,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,382-4657,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,371-7191,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,358-1921,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,375-9999,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,330-6626,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [3]:
# Data Quality Assessment
print("🔍 Data Quality Assessment")
print("="*50)

# Basic info
print(f"Total Records: {len(df_raw):,}")
print(f"Total Features: {df_raw.shape[1]}")
print(f"Memory Usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Data types
print("\n📊 Data Types:")
print(df_raw.dtypes.value_counts())

# Missing values
print("\n❌ Missing Values:")
missing_data = df_raw.isnull().sum()
missing_percent = (missing_data / len(df_raw)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

# Duplicates
duplicates = df_raw.duplicated().sum()
print(f"\n🔄 Duplicate Records: {duplicates:,} ({duplicates/len(df_raw)*100:.2f}%)")

🔍 Data Quality Assessment
Total Records: 101,174
Total Features: 17
Memory Usage: 17.95 MB

📊 Data Types:
float64    8
int64      7
object     1
bool       1
Name: count, dtype: int64

❌ Missing Values:
Empty DataFrame
Columns: [Missing Count, Missing Percentage]
Index: []

🔄 Duplicate Records: 40,729 (40.26%)


In [4]:
# Data Cleaning Process
print("🧹 Starting Data Cleaning Process...")
df_clean = df_raw.copy()

# Remove duplicates
print(f"Before duplicate removal: {len(df_clean):,} records")
df_clean = df_clean.drop_duplicates()
print(f"After duplicate removal: {len(df_clean):,} records")
print(f"Removed: {len(df_raw) - len(df_clean):,} duplicates")

# Handle missing values
numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    if df_clean[col].isnull().sum() > 0:
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)
        print(f"Filled {col} missing values with median: {median_val}")

# Convert Churn to binary
if 'Churn' in df_clean.columns:
    df_clean['Churn_Binary'] = df_clean['Churn'].map({
        'FALSE': 0, 'True': 1, False: 0, True: 1
    })
    df_clean['Churn_Binary'] = df_clean['Churn_Binary'].fillna(0).astype(int)
    print(f"Churn rate: {df_clean['Churn_Binary'].mean():.3f}")

print("\n✅ Data cleaning completed!")

🧹 Starting Data Cleaning Process...
Before duplicate removal: 101,174 records
After duplicate removal: 60,445 records
Removed: 40,729 duplicates
Churn rate: 0.110

✅ Data cleaning completed!


In [5]:
# Advanced Feature Engineering
print("🔧 Engineering Advanced Features...")
df_features = df_clean.copy()

# Ensure duplicate-free
df_features = df_features.drop_duplicates()
print(f"Re-confirmed df_features are duplicate-free: {len(df_features):,} records")

# Usage Aggregation Features
df_features['Total_Minutes'] = (df_features['Day Mins'] + df_features['Eve Mins'] + 
                               df_features['Night Mins'] + df_features['Intl Mins'])
df_features['Total_Calls'] = (df_features['Day Calls'] + df_features['Eve Calls'] + 
                             df_features['Night Calls'] + df_features['Intl Calls'])
df_features['Total_Charges'] = (df_features['Day Charge'] + df_features['Eve Charge'] + 
                               df_features['Night Charge'] + df_features['Intl Charge'])

# Usage Intensity Features
df_features['Avg_Call_Duration'] = df_features['Total_Minutes'] / (df_features['Total_Calls'] + 1e-6)
df_features['Day_Call_Duration'] = df_features['Day Mins'] / (df_features['Day Calls'] + 1e-6)
df_features['Eve_Call_Duration'] = df_features['Eve Mins'] / (df_features['Eve Calls'] + 1e-6)
df_features['Night_Call_Duration'] = df_features['Night Mins'] / (df_features['Night Calls'] + 1e-6)

# Usage Pattern Ratios
df_features['Day_Usage_Ratio'] = df_features['Day Mins'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Eve_Usage_Ratio'] = df_features['Eve Mins'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Night_Usage_Ratio'] = df_features['Night Mins'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Intl_Usage_Ratio'] = df_features['Intl Mins'] / (df_features['Total_Minutes'] + 1e-6)

# Revenue-Based Features
df_features['Revenue_Per_Minute'] = df_features['Total_Charges'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Day_Revenue_Rate'] = df_features['Day Charge'] / (df_features['Day Mins'] + 1e-6)
df_features['Eve_Revenue_Rate'] = df_features['Eve Charge'] / (df_features['Eve Mins'] + 1e-6)
df_features['Night_Revenue_Rate'] = df_features['Night Charge'] / (df_features['Night Mins'] + 1e-6)

# Customer Lifecycle Features
df_features['Account_Length_Months'] = df_features['Account Length'] / 30.0
df_features['Usage_Per_Day'] = df_features['Total_Minutes'] / (df_features['Account Length'] + 1)
df_features['Revenue_Per_Day'] = df_features['Total_Charges'] / (df_features['Account Length'] + 1)
df_features['Service_Calls_Per_Month'] = df_features['CustServ Calls'] / (df_features['Account_Length_Months'] + 1e-6)

print(f"✅ Created {df_features.shape[1] - df_clean.shape[1]} new features")
print(f"Total features now: {df_features.shape[1]}")

🔧 Engineering Advanced Features...
Re-confirmed df_features are duplicate-free: 60,445 records
✅ Created 19 new features
Total features now: 37


In [7]:
# 🎯 SMART CUSTOMER SEGMENTATION LOGIC - THE CORE OF OUR APPROACH
print("🎯 Creating Smart Customer Segmentation for Upselling...")

# Import required modules at the top
import os
import pickle
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Behavioral Indicators (Binary Flags)
df_features['Is_Heavy_Day_User'] = (df_features['Day Mins'] > df_features['Day Mins'].quantile(0.75)).astype(int)
df_features['Is_Heavy_Eve_User'] = (df_features['Eve Mins'] > df_features['Eve Mins'].quantile(0.75)).astype(int)
df_features['Is_Heavy_Night_User'] = (df_features['Night Mins'] > df_features['Night Mins'].quantile(0.75)).astype(int)
df_features['Is_Intl_User'] = (df_features['Intl Mins'] > 0).astype(int)
df_features['Is_High_Service_User'] = (df_features['CustServ Calls'] > 2).astype(int)
df_features['Has_Voicemail'] = (df_features['VMail Message'] > 0).astype(int)
df_features['Is_High_Value_Customer'] = (df_features['Total_Charges'] > df_features['Total_Charges'].quantile(0.8)).astype(int)

# 🧠 SATISFACTION SCORE - Key Innovation!
print("🧠 Calculating Customer Satisfaction Score...")
df_features['Satisfaction_Score'] = (
    (df_features['CustServ Calls'] == 0) * 0.4 +           # No complaints = very satisfied
    (df_features['Churn_Binary'] == 0) * 0.3 +             # Not churning = satisfied
    (df_features['Account Length'] > df_features['Account Length'].quantile(0.75)) * 0.2 + # Long tenure = loyal
    (df_features['Total_Minutes'] > df_features['Total_Minutes'].median()) * 0.1           # Active usage = engaged
)

# 📊 CUSTOMER VALUE SCORE (Refined)
print("📊 Calculating Customer Value Score...")
# Normalize components first
scaler = MinMaxScaler()
value_components = scaler.fit_transform(df_features[['Total_Charges', 'Account Length', 'Total_Minutes']])
df_features['Customer_Value_Score'] = (
    value_components[:, 0] * 0.4 +  # Total_Charges
    value_components[:, 1] * 0.3 +  # Account Length
    value_components[:, 2] * 0.3    # Total_Minutes
)

# ⚠️ RISK SCORE (Refined for our logic)
print("⚠️ Calculating Risk Score...")
df_features['Risk_Score'] = (
    (df_features['Churn_Binary'] == 1) * 0.5 +             # Currently churning
    (df_features['CustServ Calls'] >= 3) * 0.3 +           # High service issues
    (df_features['Total_Minutes'] < df_features['Total_Minutes'].quantile(0.25)) * 0.2  # Low usage
)

# 🎯 COMPLETELY REVISED SMART CUSTOMER CATEGORIZATION FUNCTION (Use the good one!)
def categorize_customer(row):
    satisfaction = row['Satisfaction_Score']
    value = row['Customer_Value_Score'] 
    risk = row['Risk_Score']
    account_length = row['Account Length']
    custserv_calls = row['CustServ Calls']
    churn = row['Churn_Binary']
    total_minutes = row['Total_Minutes']
    
    # CATEGORY 1: Happy Loyalists (DO_NOT_DISTURB) - Top tier satisfied customers
    if (satisfaction >= 0.6 and 
        churn == 0 and 
        custserv_calls == 0 and
        account_length > 80):
        return 'DO_NOT_DISTURB'
    
    # CATEGORY 2: At-Risk VIPs (PRIORITY_UPSELL_RETENTION) - Churning valuable customers  
    elif (churn == 1 and 
          (value >= 0.3 or account_length > 50 or total_minutes > 400)):
        return 'PRIORITY_UPSELL_RETENTION'
    
    # CATEGORY 3: Service Problem Customers (FIX_FIRST_THEN_UPSELL)
    elif (custserv_calls >= 3 and 
          churn == 0 and
          (value >= 0.2 or account_length > 30)):
        return 'FIX_FIRST_THEN_UPSELL'
    
    # CATEGORY 4: New Customers (GENTLE_UPSELL) - Recent customers with potential
    elif (account_length <= 50 and 
          churn == 0 and
          custserv_calls <= 2 and
          (satisfaction >= 0.4 or value >= 0.3)):
        return 'GENTLE_UPSELL'
    
    # CATEGORY 5: Standard Prospects (STANDARD_UPSELL) - The main target group
    elif (churn == 0 and 
          custserv_calls <= 2 and
          (satisfaction >= 0.3 or value >= 0.2 or account_length > 30)):
        return 'STANDARD_UPSELL'
    
    # CATEGORY 6: Low Priority (MINIMAL_CONTACT) - Only truly problematic customers
    else:
        return 'MINIMAL_CONTACT'

# Apply categorization
print("🔄 Applying Smart Customer Categorization...")
df_features['Customer_Category'] = df_features.apply(categorize_customer, axis=1)

# 🎯 SIMPLIFIED PRIORITY SCORE CALCULATION
def calculate_priority_score(row):
    category = row['Customer_Category']
    value = row['Customer_Value_Score'] 
    satisfaction = row['Satisfaction_Score']
    churn = row['Churn_Binary']
    custserv_calls = row['CustServ Calls']
    
    # Simplified base priorities
    if category == 'DO_NOT_DISTURB':
        return max(1, 8 - (satisfaction * 5))  # 1-8 range
    elif category == 'PRIORITY_UPSELL_RETENTION':
        return min(100, 85 + (value * 30))     # 85-100 range
    elif category == 'FIX_FIRST_THEN_UPSELL':
        return min(85, 70 + (custserv_calls * 3))  # 70-85 range
    elif category == 'GENTLE_UPSELL':
        return min(70, 50 + (value * 40))      # 50-70 range
    elif category == 'STANDARD_UPSELL':
        return min(65, 30 + (value * 50))      # 30-65 range
    else:  # MINIMAL_CONTACT
        return min(25, 10 + (value * 30))      # 10-25 range

# Apply priority calculation
print("📊 Calculating Priority Scores...")
df_features['Priority_Score'] = df_features.apply(calculate_priority_score, axis=1)

# 🏷️ PRIORITY LEVEL ASSIGNMENT
def assign_priority_level(score):
    if score >= 90:
        return 'CRITICAL'      # Immediate action required
    elif score >= 70:
        return 'HIGH'          # Action within 1 week
    elif score >= 50:
        return 'MEDIUM'        # Action within 1 month
    elif score >= 25:
        return 'LOW'           # Action within quarter
    else:
        return 'MINIMAL'       # Annual review only

df_features['Priority_Level'] = df_features['Priority_Score'].apply(assign_priority_level)

# 🔢 ENCODE CATEGORIES FOR ML
print("🔢 Encoding Categories for Machine Learning...")
label_encoder = LabelEncoder()
df_features['Upsell_Priority_Encoded'] = label_encoder.fit_transform(df_features['Customer_Category'])

# Save the label encoder for later use
os.makedirs('../../models/trained_models', exist_ok=True)
with open('../../models/trained_models/upsell_priority_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Engagement metrics (refined)
df_features['Engagement_Score'] = (
    df_features['Has_Voicemail'] * 0.2 +
    df_features['Is_Intl_User'] * 0.3 +
    (df_features['CustServ Calls'] == 0) * 0.5
)

print(f"✅ Smart Customer Segmentation Completed!")
print(f"Total features after smart engineering: {df_features.shape[1]}")


🎯 Creating Smart Customer Segmentation for Upselling...
🧠 Calculating Customer Satisfaction Score...
📊 Calculating Customer Value Score...
⚠️ Calculating Risk Score...
🔄 Applying Smart Customer Categorization...
📊 Calculating Priority Scores...
🔢 Encoding Categories for Machine Learning...
✅ Smart Customer Segmentation Completed!
Total features after smart engineering: 52


In [10]:
# 🎯 COMPLETELY REVISED SMART CUSTOMER CATEGORIZATION FUNCTION
def categorize_customer(row):
    satisfaction = row['Satisfaction_Score']
    value = row['Customer_Value_Score'] 
    risk = row['Risk_Score']
    account_length = row['Account Length']
    custserv_calls = row['CustServ Calls']
    churn = row['Churn_Binary']
    total_minutes = row['Total_Minutes']
    
    # CATEGORY 1: Happy Loyalists (DO_NOT_DISTURB) - Top tier satisfied customers
    if (satisfaction >= 0.6 and 
        churn == 0 and 
        custserv_calls == 0 and
        account_length > 80):
        return 'DO_NOT_DISTURB'
    
    # CATEGORY 2: At-Risk VIPs (PRIORITY_UPSELL_RETENTION) - Churning valuable customers  
    elif (churn == 1 and 
          (value >= 0.3 or account_length > 50 or total_minutes > 400)):
        return 'PRIORITY_UPSELL_RETENTION'
    
    # CATEGORY 3: Service Problem Customers (FIX_FIRST_THEN_UPSELL)
    elif (custserv_calls >= 3 and 
          churn == 0 and
          (value >= 0.2 or account_length > 30)):
        return 'FIX_FIRST_THEN_UPSELL'
    
    # CATEGORY 4: New Customers (GENTLE_UPSELL) - Recent customers with potential
    elif (account_length <= 50 and 
          churn == 0 and
          custserv_calls <= 2 and
          (satisfaction >= 0.4 or value >= 0.3)):
        return 'GENTLE_UPSELL'
    
    # CATEGORY 5: Standard Prospects (STANDARD_UPSELL) - The main target group
    elif (churn == 0 and 
          custserv_calls <= 2 and
          (satisfaction >= 0.3 or value >= 0.2 or account_length > 30)):
        return 'STANDARD_UPSELL'
    
    # CATEGORY 6: Low Priority (MINIMAL_CONTACT) - Only truly problematic customers
    else:
        return 'MINIMAL_CONTACT'

# Apply the COMPLETELY REVISED categorization
print("🔄 Applying COMPLETELY REVISED Smart Customer Categorization...")
df_features['Customer_Category'] = df_features.apply(categorize_customer, axis=1)

# 🎯 SIMPLIFIED PRIORITY SCORE CALCULATION
def calculate_priority_score(row):
    category = row['Customer_Category']
    value = row['Customer_Value_Score'] 
    satisfaction = row['Satisfaction_Score']
    churn = row['Churn_Binary']
    custserv_calls = row['CustServ Calls']
    
    # Simplified base priorities
    if category == 'DO_NOT_DISTURB':
        return max(1, 8 - (satisfaction * 5))  # 1-8 range
    elif category == 'PRIORITY_UPSELL_RETENTION':
        return min(100, 85 + (value * 30))     # 85-100 range
    elif category == 'FIX_FIRST_THEN_UPSELL':
        return min(85, 70 + (custserv_calls * 3))  # 70-85 range
    elif category == 'GENTLE_UPSELL':
        return min(70, 50 + (value * 40))      # 50-70 range
    elif category == 'STANDARD_UPSELL':
        return min(65, 30 + (value * 50))      # 30-65 range
    else:  # MINIMAL_CONTACT
        return min(25, 10 + (value * 30))      # 10-25 range

# Apply REVISED priority calculation
print("📊 Calculating REVISED Priority Scores...")
df_features['Priority_Score'] = df_features.apply(calculate_priority_score, axis=1)
df_features['Priority_Level'] = df_features['Priority_Score'].apply(assign_priority_level)

# 📊 VALIDATION WITH COMPLETELY REVISED LOGIC
print("📊 COMPLETELY REVISED Smart Customer Segmentation Results")
print("="*60)

# Customer Category Distribution
print("\n🎯 REVISED CUSTOMER CATEGORY DISTRIBUTION:")
category_counts = df_features['Customer_Category'].value_counts()
category_percentages = df_features['Customer_Category'].value_counts(normalize=True) * 100

for category in ['DO_NOT_DISTURB', 'STANDARD_UPSELL', 'GENTLE_UPSELL', 
                'PRIORITY_UPSELL_RETENTION', 'FIX_FIRST_THEN_UPSELL', 'MINIMAL_CONTACT']:
    if category in category_counts.index:
        count = category_counts[category]
        percentage = category_percentages[category]
        print(f"  {category}: {count:,} customers ({percentage:.1f}%)")

# Priority Level Distribution
print("\n📈 REVISED PRIORITY LEVEL DISTRIBUTION:")
priority_counts = df_features['Priority_Level'].value_counts()
for level in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW', 'MINIMAL']:
    if level in priority_counts.index:
        count = priority_counts[level]
        percentage = (count / len(df_features)) * 100
        print(f"  {level}: {count:,} customers ({percentage:.1f}%)")

# Key Business Insights
do_not_disturb_count = len(df_features[df_features['Customer_Category'] == 'DO_NOT_DISTURB'])
priority_upsell_count = len(df_features[df_features['Customer_Category'] == 'PRIORITY_UPSELL_RETENTION'])
standard_upsell_count = len(df_features[df_features['Customer_Category'] == 'STANDARD_UPSELL'])
gentle_upsell_count = len(df_features[df_features['Customer_Category'] == 'GENTLE_UPSELL'])
fix_first_count = len(df_features[df_features['Customer_Category'] == 'FIX_FIRST_THEN_UPSELL'])
minimal_contact_count = len(df_features[df_features['Customer_Category'] == 'MINIMAL_CONTACT'])
total_customers = len(df_features)

print(f"\n💡 REVISED BUSINESS INSIGHTS:")
print(f"  📊 Total Customers: {total_customers:,}")
print(f"  😊 Happy Loyalists (DO_NOT_DISTURB): {do_not_disturb_count:,} ({do_not_disturb_count/total_customers*100:.1f}%)")
print(f"  🎯 Standard Upsell Targets: {standard_upsell_count:,} ({standard_upsell_count/total_customers*100:.1f}%)")
print(f"  🌱 Gentle Upsell (New Customers): {gentle_upsell_count:,} ({gentle_upsell_count/total_customers*100:.1f}%)")
print(f"  🚨 Priority Retention Upsell: {priority_upsell_count:,} ({priority_upsell_count/total_customers*100:.1f}%)")
print(f"  🔧 Fix First Then Upsell: {fix_first_count:,} ({fix_first_count/total_customers*100:.1f}%)")
print(f"  ⚠️ Minimal Contact: {minimal_contact_count:,} ({minimal_contact_count/total_customers*100:.1f}%)")

total_upsell_targets = standard_upsell_count + gentle_upsell_count + priority_upsell_count + fix_first_count
print(f"\n🎯 TOTAL ACTIONABLE CUSTOMERS: {total_upsell_targets:,} ({total_upsell_targets/total_customers*100:.1f}%)")
print(f"✅ CUSTOMERS TO PRESERVE: {do_not_disturb_count:,} ({do_not_disturb_count/total_customers*100:.1f}%)")

# Success Check
if minimal_contact_count / total_customers <= 0.3:  # 30% or less
    print(f"\n✅ SUCCESS: Balanced distribution achieved!")
    print(f"📈 {total_upsell_targets/total_customers*100:.1f}% of customers are actionable upsell targets")
    print("🚀 Ready for business deployment!")
else:
    print(f"\n⚠️ Still needs adjustment: {minimal_contact_count/total_customers*100:.1f}% in MINIMAL_CONTACT")
    
    # Emergency fallback - make STANDARD_UPSELL even more inclusive
    if minimal_contact_count / total_customers > 0.5:
        print("🔧 Applying emergency fallback logic...")
        
        def emergency_categorize(row):
            churn = row['Churn_Binary']
            custserv_calls = row['CustServ Calls']
            satisfaction = row['Satisfaction_Score']
            account_length = row['Account Length']
            
            if (satisfaction >= 0.6 and churn == 0 and custserv_calls == 0):
                return 'DO_NOT_DISTURB'
            elif (churn == 1):
                return 'PRIORITY_UPSELL_RETENTION'
            elif (custserv_calls >= 3 and churn == 0):
                return 'FIX_FIRST_THEN_UPSELL'
            elif (account_length <= 50 and churn == 0):
                return 'GENTLE_UPSELL'
            elif (churn == 0):  # All non-churning customers become standard upsell
                return 'STANDARD_UPSELL'
            else:
                return 'MINIMAL_CONTACT'
        
        df_features['Customer_Category'] = df_features.apply(emergency_categorize, axis=1)
        df_features['Priority_Score'] = df_features.apply(calculate_priority_score, axis=1)
        df_features['Priority_Level'] = df_features['Priority_Score'].apply(assign_priority_level)
        
        print("🔄 Emergency categorization applied!")
        
        # Show final results
        final_category_counts = df_features['Customer_Category'].value_counts()
        final_category_percentages = df_features['Customer_Category'].value_counts(normalize=True) * 100
        
        print("\n🎯 FINAL EMERGENCY DISTRIBUTION:")
        for category in final_category_counts.index:
            count = final_category_counts[category]
            percentage = final_category_percentages[category]
            print(f"  {category}: {count:,} customers ({percentage:.1f}%)")


🔄 Applying COMPLETELY REVISED Smart Customer Categorization...
📊 Calculating REVISED Priority Scores...
📊 COMPLETELY REVISED Smart Customer Segmentation Results

🎯 REVISED CUSTOMER CATEGORY DISTRIBUTION:
  DO_NOT_DISTURB: 8,238 customers (13.6%)
  STANDARD_UPSELL: 26,751 customers (44.3%)
  GENTLE_UPSELL: 3,623 customers (6.0%)
  PRIORITY_UPSELL_RETENTION: 6,676 customers (11.0%)
  FIX_FIRST_THEN_UPSELL: 14,248 customers (23.6%)
  MINIMAL_CONTACT: 909 customers (1.5%)

📈 REVISED PRIORITY LEVEL DISTRIBUTION:
  CRITICAL: 182 customers (0.3%)
  HIGH: 20,742 customers (34.3%)
  MEDIUM: 3,671 customers (6.1%)
  LOW: 26,703 customers (44.2%)
  MINIMAL: 9,147 customers (15.1%)

💡 REVISED BUSINESS INSIGHTS:
  📊 Total Customers: 60,445
  😊 Happy Loyalists (DO_NOT_DISTURB): 8,238 (13.6%)
  🎯 Standard Upsell Targets: 26,751 (44.3%)
  🌱 Gentle Upsell (New Customers): 3,623 (6.0%)
  🚨 Priority Retention Upsell: 6,676 (11.0%)
  🔧 Fix First Then Upsell: 14,248 (23.6%)
  ⚠️ Minimal Contact: 909 (1.5%)

In [12]:
# 💾 Save processed data with smart segmentation
print("💾 Saving Smart Segmentation Results...")

# Create directories
import os
import json
os.makedirs('../../data/processed', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)  # For notebook access

# Save the complete processed dataset
df_features.to_csv('../../data/processed/telecom_processed.csv', index=False)
df_features.to_csv('../data/processed/telecom_processed.csv', index=False)  # For notebook access
print(f"✅ Saved complete processed dataset: {df_features.shape}")

# Save feature names for ML pipeline
feature_names = list(df_features.columns)
with open('../../data/processed/feature_names.txt', 'w') as f:
    for feature in feature_names:
        f.write(f"{feature}\n")
        
with open('../data/processed/feature_names.txt', 'w') as f:
    for feature in feature_names:
        f.write(f"{feature}\n")

# Save customer segments for business use
os.makedirs('../../outputs/exports/customer_segments', exist_ok=True)

# Export priority customers for immediate action
priority_customers = df_features[df_features['Customer_Category'] == 'PRIORITY_UPSELL_RETENTION']
priority_customers[['Phone Number', 'Customer_Category', 'Priority_Score', 'Priority_Level', 
                   'Customer_Value_Score', 'Satisfaction_Score', 'Risk_Score', 'CustServ Calls',
                   'Account Length', 'Total_Charges']].to_csv(
    '../../outputs/exports/customer_segments/priority_tier1_customers.csv', index=False)

# Export happy loyalists to NOT disturb
happy_loyalists = df_features[df_features['Customer_Category'] == 'DO_NOT_DISTURB']
happy_loyalists[['Phone Number', 'Customer_Category', 'Priority_Score', 'Satisfaction_Score',
                'Customer_Value_Score', 'Account Length', 'Total_Charges']].to_csv(
    '../../outputs/exports/customer_segments/do_not_disturb_loyalists.csv', index=False)

# Export complete priority master list
df_features[['Phone Number', 'Customer_Category', 'Priority_Score', 'Priority_Level',
            'Customer_Value_Score', 'Satisfaction_Score', 'Risk_Score', 'Upsell_Priority_Encoded']].to_csv(
    '../../outputs/exports/customer_segments/customer_priority_master.csv', index=False)

# 🔧 FIXED: Save category mapping for reference (convert numpy int64 to regular int)
category_mapping = {int(k): v for k, v in zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_)}
with open('../../models/trained_models/category_mapping.json', 'w') as f:
    json.dump(category_mapping, f, indent=2)

print(f"\n📊 EXPORT SUMMARY:")
print(f"  🚨 Priority Tier 1 Customers: {len(priority_customers):,}")
print(f"  😊 Happy Loyalists (Do Not Disturb): {len(happy_loyalists):,}")
print(f"  📋 Complete Customer Master List: {len(df_features):,}")

print(f"\n✅ Saved {len(feature_names)} feature names")
print(f"✅ Saved label encoder and category mapping")
print(f"✅ Exported customer segments for business use")
print("\n🎉 Smart Customer Segmentation Preprocessing Completed Successfully!")
print("\n🎯 NEXT STEPS:")
print("  1. Run EDA notebook to validate segments")
print("  2. Train multi-class models on 'Upsell_Priority_Encoded'")
print("  3. Deploy smart upselling system")
print("\n💡 KEY INSIGHT: We've transformed churn prediction into intelligent customer prioritization!")


💾 Saving Smart Segmentation Results...
✅ Saved complete processed dataset: (60445, 52)

📊 EXPORT SUMMARY:
  🚨 Priority Tier 1 Customers: 6,676
  😊 Happy Loyalists (Do Not Disturb): 8,238
  📋 Complete Customer Master List: 60,445

✅ Saved 52 feature names
✅ Saved label encoder and category mapping
✅ Exported customer segments for business use

🎉 Smart Customer Segmentation Preprocessing Completed Successfully!

🎯 NEXT STEPS:
  1. Run EDA notebook to validate segments
  2. Train multi-class models on 'Upsell_Priority_Encoded'
  3. Deploy smart upselling system

💡 KEY INSIGHT: We've transformed churn prediction into intelligent customer prioritization!


In [13]:
# 📊 VIEW ALL SAVED CONTENT FROM SMART SEGMENTATION
print("📊 VIEWING SAVED SMART SEGMENTATION CONTENT")
print("="*60)

import pandas as pd
import json
import pickle
import os

# 1. 📋 VIEW PROCESSED DATASET SUMMARY
print("\n1. 📋 PROCESSED DATASET SUMMARY:")
try:
    df_processed = pd.read_csv('../../data/processed/telecom_processed.csv')
    print(f"   Shape: {df_processed.shape}")
    print(f"   Columns: {len(df_processed.columns)}")
    print(f"   Memory Usage: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Show new smart features
    smart_features = ['Satisfaction_Score', 'Customer_Value_Score', 'Risk_Score', 
                     'Customer_Category', 'Priority_Score', 'Priority_Level', 'Upsell_Priority_Encoded']
    print(f"\n   🎯 Smart Segmentation Features:")
    for feature in smart_features:
        if feature in df_processed.columns:
            print(f"     ✅ {feature}")
        else:
            print(f"     ❌ {feature} (missing)")
            
    print(f"\n   📊 Customer Category Distribution:")
    if 'Customer_Category' in df_processed.columns:
        category_dist = df_processed['Customer_Category'].value_counts()
        for category, count in category_dist.items():
            percentage = (count / len(df_processed)) * 100
            print(f"     {category}: {count:,} ({percentage:.1f}%)")
    
except Exception as e:
    print(f"   ❌ Error loading processed dataset: {e}")

# 2. 📝 VIEW FEATURE NAMES
print("\n2. 📝 FEATURE NAMES LIST:")
try:
    with open('../../data/processed/feature_names.txt', 'r') as f:
        features = f.read().strip().split('\n')
    print(f"   Total Features: {len(features)}")
    print(f"   First 10 features: {features[:10]}")
    print(f"   Last 10 features: {features[-10:]}")
except Exception as e:
    print(f"   ❌ Error loading feature names: {e}")

# 3. 🚨 VIEW PRIORITY TIER 1 CUSTOMERS (Critical Action Needed)
print("\n3. 🚨 PRIORITY TIER 1 CUSTOMERS (IMMEDIATE ACTION):")
try:
    priority_customers = pd.read_csv('../../outputs/exports/customer_segments/priority_tier1_customers.csv')
    print(f"   Count: {len(priority_customers):,} customers")
    print(f"   Columns: {list(priority_customers.columns)}")
    if len(priority_customers) > 0:
        print(f"\n   📊 Sample Priority Customers:")
        print(priority_customers.head(3)[['Phone Number', 'Priority_Score', 'Customer_Value_Score', 
                                         'Risk_Score', 'CustServ Calls', 'Account Length']].to_string(index=False))
        
        print(f"\n   📈 Priority Statistics:")
        print(f"     Average Priority Score: {priority_customers['Priority_Score'].mean():.1f}")
        print(f"     Average Customer Value: {priority_customers['Customer_Value_Score'].mean():.3f}")
        print(f"     Average Service Calls: {priority_customers['CustServ Calls'].mean():.1f}")
        print(f"     Average Account Length: {priority_customers['Account Length'].mean():.0f} days")
except Exception as e:
    print(f"   ❌ Error loading priority customers: {e}")

# 4. 😊 VIEW HAPPY LOYALISTS (Do Not Disturb)
print("\n4. 😊 HAPPY LOYALISTS (DO NOT DISTURB):")
try:
    happy_loyalists = pd.read_csv('../../outputs/exports/customer_segments/do_not_disturb_loyalists.csv')
    print(f"   Count: {len(happy_loyalists):,} customers")
    if len(happy_loyalists) > 0:
        print(f"\n   📊 Sample Happy Loyalists:")
        print(happy_loyalists.head(3)[['Phone Number', 'Priority_Score', 'Satisfaction_Score', 
                                      'Customer_Value_Score', 'Account Length']].to_string(index=False))
        
        print(f"\n   📈 Loyalty Statistics:")
        print(f"     Average Satisfaction Score: {happy_loyalists['Satisfaction_Score'].mean():.3f}")
        print(f"     Average Priority Score: {happy_loyalists['Priority_Score'].mean():.1f}")
        print(f"     Average Account Length: {happy_loyalists['Account Length'].mean():.0f} days")
except Exception as e:
    print(f"   ❌ Error loading happy loyalists: {e}")

# 5. 📋 VIEW COMPLETE CUSTOMER PRIORITY MASTER LIST
print("\n5. 📋 CUSTOMER PRIORITY MASTER LIST:")
try:
    master_list = pd.read_csv('../../outputs/exports/customer_segments/customer_priority_master.csv')
    print(f"   Total Customers: {len(master_list):,}")
    print(f"   Columns: {list(master_list.columns)}")
    
    print(f"\n   📊 Priority Level Distribution:")
    if 'Priority_Level' in master_list.columns:
        priority_dist = master_list['Priority_Level'].value_counts()
        for level in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW', 'MINIMAL']:
            if level in priority_dist.index:
                count = priority_dist[level]
                percentage = (count / len(master_list)) * 100
                print(f"     {level}: {count:,} ({percentage:.1f}%)")
    
    print(f"\n   🎯 Top 5 Highest Priority Customers:")
    top_priority = master_list.nlargest(5, 'Priority_Score')
    print(top_priority[['Phone Number', 'Customer_Category', 'Priority_Score', 'Priority_Level']].to_string(index=False))
    
except Exception as e:
    print(f"   ❌ Error loading master list: {e}")

# 6. 🔢 VIEW CATEGORY MAPPING
print("\n6. 🔢 CATEGORY MAPPING (For ML Models):")
try:
    with open('../../models/trained_models/category_mapping.json', 'r') as f:
        category_mapping = json.load(f)
    print(f"   Encoded Categories:")
    for code, category in category_mapping.items():
        print(f"     {code}: {category}")
except Exception as e:
    print(f"   ❌ Error loading category mapping: {e}")

# 7. 🤖 VIEW LABEL ENCODER
print("\n7. 🤖 LABEL ENCODER (For Production):")
try:
    with open('../../models/trained_models/upsell_priority_label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    print(f"   Classes: {list(label_encoder.classes_)}")
    print(f"   Number of Classes: {len(label_encoder.classes_)}")
    print(f"   Example Encoding:")
    for i, class_name in enumerate(label_encoder.classes_):
        print(f"     '{class_name}' → {i}")
except Exception as e:
    print(f"   ❌ Error loading label encoder: {e}")

# 8. 📁 VIEW DIRECTORY STRUCTURE
print("\n8. 📁 CREATED DIRECTORY STRUCTURE:")
directories = [
    '../../data/processed',
    '../../outputs/exports/customer_segments',
    '../../models/trained_models'
]

for directory in directories:
    print(f"\n   📂 {directory}:")
    try:
        if os.path.exists(directory):
            files = os.listdir(directory)
            for file in files:
                file_path = os.path.join(directory, file)
                if os.path.isfile(file_path):
                    size = os.path.getsize(file_path) / 1024  # KB
                    print(f"     📄 {file} ({size:.1f} KB)")
        else:
            print(f"     ❌ Directory does not exist")
    except Exception as e:
        print(f"     ❌ Error accessing directory: {e}")

# 9. 💡 BUSINESS INSIGHTS SUMMARY
print("\n9. 💡 BUSINESS INSIGHTS SUMMARY:")
try:
    df = pd.read_csv('../../data/processed/telecom_processed.csv')
    
    total_customers = len(df)
    actionable_customers = len(df[df['Customer_Category'].isin(['STANDARD_UPSELL', 'GENTLE_UPSELL', 
                                                               'PRIORITY_UPSELL_RETENTION', 'FIX_FIRST_THEN_UPSELL'])])
    preserve_customers = len(df[df['Customer_Category'] == 'DO_NOT_DISTURB'])
    critical_customers = len(df[df['Priority_Level'] == 'CRITICAL'])
    
    print(f"   📊 Total Customers: {total_customers:,}")
    print(f"   🎯 Actionable for Upselling: {actionable_customers:,} ({actionable_customers/total_customers*100:.1f}%)")
    print(f"   😊 Preserve Relationships: {preserve_customers:,} ({preserve_customers/total_customers*100:.1f}%)")
    print(f"   🚨 Critical Action Needed: {critical_customers:,} ({critical_customers/total_customers*100:.1f}%)")
    
    # Revenue potential
    high_value_actionable = len(df[(df['Customer_Category'].isin(['STANDARD_UPSELL', 'GENTLE_UPSELL', 
                                                                 'PRIORITY_UPSELL_RETENTION', 'FIX_FIRST_THEN_UPSELL'])) & 
                                  (df['Customer_Value_Score'] > 0.5)])
    
    print(f"   💰 High-Value Actionable Customers: {high_value_actionable:,}")
    print(f"   📈 Estimated Revenue Opportunity: {actionable_customers * 50:,} USD (assuming $50 avg upsell)")
    
except Exception as e:
    print(f"   ❌ Error calculating business insights: {e}")

print(f"\n🎉 SMART SEGMENTATION CONTENT REVIEW COMPLETED!")
print(f"🚀 Ready for next phase: EDA validation and model training!")


📊 VIEWING SAVED SMART SEGMENTATION CONTENT

1. 📋 PROCESSED DATASET SUMMARY:
   Shape: (60445, 52)
   Columns: 52
   Memory Usage: 33.75 MB

   🎯 Smart Segmentation Features:
     ✅ Satisfaction_Score
     ✅ Customer_Value_Score
     ✅ Risk_Score
     ✅ Customer_Category
     ✅ Priority_Score
     ✅ Priority_Level
     ✅ Upsell_Priority_Encoded

   📊 Customer Category Distribution:
     STANDARD_UPSELL: 26,751 (44.3%)
     FIX_FIRST_THEN_UPSELL: 14,248 (23.6%)
     DO_NOT_DISTURB: 8,238 (13.6%)
     PRIORITY_UPSELL_RETENTION: 6,676 (11.0%)
     GENTLE_UPSELL: 3,623 (6.0%)
     MINIMAL_CONTACT: 909 (1.5%)

2. 📝 FEATURE NAMES LIST:
   Total Features: 52
   First 10 features: ['Phone Number', 'Account Length', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins']
   Last 10 features: ['Has_Voicemail', 'Is_High_Value_Customer', 'Satisfaction_Score', 'Customer_Value_Score', 'Risk_Score', 'Customer_Category', 'Priority_Score', 'Priority_L

In [14]:
# 🎯 CREATE REVISED OUTPUT FORMAT WITH SMART SEGMENTATION
print("🎯 Creating Revised Output Format with Smart Upselling Logic...")

# Create the enhanced output DataFrame
output_df = df_features.copy()

# 🔧 ADD SMART UPSELLING LOGIC COLUMNS

# 1. Upsell_Probability - Based on our smart categories
def calculate_upsell_probability(row):
    category = row['Customer_Category']
    value = row['Customer_Value_Score']
    satisfaction = row['Satisfaction_Score']
    risk = row['Risk_Score']
    
    if category == 'DO_NOT_DISTURB':
        return 0.05  # Very low probability - don't upsell
    elif category == 'PRIORITY_UPSELL_RETENTION':
        return min(0.95, 0.75 + (value * 0.4))  # High probability for retention
    elif category == 'FIX_FIRST_THEN_UPSELL':
        return min(0.85, 0.60 + (value * 0.5))  # Good probability after fixing issues
    elif category == 'GENTLE_UPSELL':
        return min(0.70, 0.45 + (satisfaction * 0.5))  # Moderate probability
    elif category == 'STANDARD_UPSELL':
        return min(0.75, 0.35 + (value * 0.6))  # Standard probability
    else:  # MINIMAL_CONTACT
        return 0.10  # Very low probability

output_df['Upsell_Probability'] = output_df.apply(calculate_upsell_probability, axis=1)

# 2. Upsell_Prediction - Binary decision based on probability and category
def determine_upsell_prediction(row):
    category = row['Customer_Category']
    probability = row['Upsell_Probability']
    
    if category == 'DO_NOT_DISTURB':
        return 0  # Never upsell happy loyalists
    elif category == 'MINIMAL_CONTACT':
        return 0  # Don't upsell problematic customers
    else:
        return 1 if probability >= 0.3 else 0

output_df['Upsell_Prediction'] = output_df.apply(determine_upsell_prediction, axis=1)

# 3. Confidence_Level - Based on our segmentation confidence
def calculate_confidence_level(row):
    category = row['Customer_Category']
    satisfaction = row['Satisfaction_Score']
    value = row['Customer_Value_Score']
    
    if category == 'DO_NOT_DISTURB':
        return 'Very High' if satisfaction >= 0.8 else 'High'
    elif category == 'PRIORITY_UPSELL_RETENTION':
        return 'Very High' if value >= 0.5 else 'High'
    elif category == 'FIX_FIRST_THEN_UPSELL':
        return 'High'
    elif category in ['GENTLE_UPSELL', 'STANDARD_UPSELL']:
        return 'Medium' if value >= 0.4 else 'Low'
    else:
        return 'Low'

output_df['Confidence_Level'] = output_df.apply(calculate_confidence_level, axis=1)

# 4. Recommended_Product - Tailored by segment
def recommend_product(row):
    category = row['Customer_Category']
    value = row['Customer_Value_Score']
    custserv_calls = row['CustServ Calls']
    
    if category == 'DO_NOT_DISTURB':
        return 'Loyalty Appreciation Package'
    elif category == 'PRIORITY_UPSELL_RETENTION':
        return 'Emergency Retention Package'
    elif category == 'FIX_FIRST_THEN_UPSELL':
        return 'Premium Support + Service Upgrade'
    elif category == 'GENTLE_UPSELL':
        return 'Starter Premium Features'
    elif category == 'STANDARD_UPSELL':
        if value >= 0.5:
            return 'Premium Upgrade Package'
        else:
            return 'Standard Upgrade Package'
    else:  # MINIMAL_CONTACT
        return 'Basic Service Review'

output_df['Recommended_Product'] = output_df.apply(recommend_product, axis=1)

# 5. Expected_Monthly_Revenue - Based on category and value
def calculate_expected_revenue(row):
    category = row['Customer_Category']
    value = row['Customer_Value_Score']
    probability = row['Upsell_Probability']
    current_charges = row['Total_Charges']
    
    # Base revenue multipliers by category
    if category == 'DO_NOT_DISTURB':
        base_multiplier = 0.0  # No additional revenue expected
    elif category == 'PRIORITY_UPSELL_RETENTION':
        base_multiplier = 1.5  # High retention value
    elif category == 'FIX_FIRST_THEN_UPSELL':
        base_multiplier = 1.2  # Moderate increase after fixing issues
    elif category == 'GENTLE_UPSELL':
        base_multiplier = 0.8  # Conservative increase for new customers
    elif category == 'STANDARD_UPSELL':
        base_multiplier = 1.0  # Standard increase
    else:  # MINIMAL_CONTACT
        base_multiplier = 0.1  # Minimal expected revenue
    
    # Calculate expected revenue
    base_revenue = current_charges * base_multiplier
    expected_revenue = base_revenue * probability * (1 + value)
    
    return round(expected_revenue, 2)

output_df['Expected_Monthly_Revenue'] = output_df.apply(calculate_expected_revenue, axis=1)

# 6. Priority - Convert our priority levels to match your format
def convert_priority(row):
    level = row['Priority_Level']
    category = row['Customer_Category']
    
    if category == 'DO_NOT_DISTURB':
        return 'DO NOT CONTACT'
    elif level == 'CRITICAL':
        return 'VERY HIGH'
    elif level == 'HIGH':
        return 'HIGH'
    elif level == 'MEDIUM':
        return 'MEDIUM'
    elif level == 'LOW':
        return 'LOW'
    else:
        return 'MINIMAL'

output_df['Priority'] = output_df.apply(convert_priority, axis=1)

# 7. Customer_Segment - Human-readable segment descriptions
def create_segment_description(row):
    category = row['Customer_Category']
    value = row['Customer_Value_Score']
    satisfaction = row['Satisfaction_Score']
    
    segment_map = {
        'DO_NOT_DISTURB': 'Happy Loyalists (Preserve Relationship)',
        'PRIORITY_UPSELL_RETENTION': 'At-Risk VIPs (Save Immediately)',
        'FIX_FIRST_THEN_UPSELL': 'Problem Solvers (Fix Issues First)',
        'GENTLE_UPSELL': 'New Prospects (Gentle Approach)',
        'STANDARD_UPSELL': 'Regular Prospects (Standard Campaign)',
        'MINIMAL_CONTACT': 'Low Priority (Minimal Contact)'
    }
    
    return segment_map.get(category, 'Unknown Segment')

output_df['Customer_Segment'] = output_df.apply(create_segment_description, axis=1)

# 📊 SELECT FINAL OUTPUT COLUMNS (matching your sample structure)
final_columns = [
    'Phone Number', 'Account Length', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
    'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge',
    'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn',
    'Upsell_Probability', 'Upsell_Prediction', 'Confidence_Level', 'Recommended_Product',
    'Expected_Monthly_Revenue', 'Priority', 'Customer_Segment'
]

# Create final output DataFrame
final_output = output_df[final_columns].copy()

# 💾 SAVE THE REVISED OUTPUT FORMAT
print("💾 Saving Revised Output Format...")

# Save complete output
final_output.to_csv('../../outputs/exports/customer_segments/smart_upsell_predictions.csv', index=False)

# Save sample (first 1000 rows) for review
sample_output = final_output.head(1000)
sample_output.to_csv('../../outputs/exports/customer_segments/smart_upsell_sample.csv', index=False)

print(f"✅ Saved complete smart upsell predictions: {len(final_output):,} customers")
print(f"✅ Saved sample output: {len(sample_output):,} customers")

# 📊 SHOW SAMPLE OF REVISED OUTPUT
print("\n📊 SAMPLE OF REVISED OUTPUT FORMAT:")
print("="*80)

# Show first 10 rows with key columns
display_columns = ['Phone Number', 'CustServ Calls', 'Churn', 'Upsell_Probability', 
                  'Upsell_Prediction', 'Confidence_Level', 'Recommended_Product', 
                  'Expected_Monthly_Revenue', 'Priority', 'Customer_Segment']

sample_display = final_output[display_columns].head(10)
print(sample_display.to_string(index=False))

# 📈 SUMMARY STATISTICS
print("\n📈 REVISED OUTPUT SUMMARY:")
print("="*50)

print(f"📊 Total Customers: {len(final_output):,}")
print(f"📊 Total Features: {len(final_output.columns)}")

print("\n🎯 UPSELL PREDICTIONS:")
upsell_dist = final_output['Upsell_Prediction'].value_counts()
print(f"  ✅ Recommended for Upsell: {upsell_dist.get(1, 0):,} ({upsell_dist.get(1, 0)/len(final_output)*100:.1f}%)")
print(f"  ❌ Not Recommended: {upsell_dist.get(0, 0):,} ({upsell_dist.get(0, 0)/len(final_output)*100:.1f}%)")

print("\n🎯 PRIORITY DISTRIBUTION:")
priority_dist = final_output['Priority'].value_counts()
for priority in ['VERY HIGH', 'HIGH', 'MEDIUM', 'LOW', 'DO NOT CONTACT', 'MINIMAL']:
    if priority in priority_dist.index:
        count = priority_dist[priority]
        percentage = (count / len(final_output)) * 100
        print(f"  {priority}: {count:,} ({percentage:.1f}%)")

print("\n💰 REVENUE PROJECTIONS:")
total_expected_revenue = final_output['Expected_Monthly_Revenue'].sum()
avg_expected_revenue = final_output[final_output['Upsell_Prediction'] == 1]['Expected_Monthly_Revenue'].mean()
print(f"  Total Expected Monthly Revenue: ${total_expected_revenue:,.2f}")
print(f"  Average Revenue per Upsell Target: ${avg_expected_revenue:.2f}")

print("\n🎯 CONFIDENCE LEVELS:")
confidence_dist = final_output['Confidence_Level'].value_counts()
for conf in ['Very High', 'High', 'Medium', 'Low']:
    if conf in confidence_dist.index:
        count = confidence_dist[conf]
        percentage = (count / len(final_output)) * 100
        print(f"  {conf}: {count:,} ({percentage:.1f}%)")

print("\n🎉 REVISED OUTPUT FORMAT COMPLETED!")
print("🚀 Your smart upselling predictions are ready for business deployment!")
print("\n📁 Files Created:")
print("  📄 smart_upsell_predictions.csv - Complete dataset")
print("  📄 smart_upsell_sample.csv - Sample for review")
print("\n💡 KEY INSIGHT: This output respects customer satisfaction while maximizing business opportunities!")

🎯 Creating Revised Output Format with Smart Upselling Logic...
💾 Saving Revised Output Format...
✅ Saved complete smart upsell predictions: 60,445 customers
✅ Saved sample output: 1,000 customers

📊 SAMPLE OF REVISED OUTPUT FORMAT:
Phone Number  CustServ Calls  Churn  Upsell_Probability  Upsell_Prediction Confidence_Level               Recommended_Product  Expected_Monthly_Revenue       Priority                        Customer_Segment
    382-4657               1  False            0.360622                  1              Low          Standard Upgrade Package                     27.73            LOW   Regular Prospects (Standard Campaign)
    371-7191               1  False            0.357735                  1              Low          Standard Upgrade Package                     21.47            LOW   Regular Prospects (Standard Campaign)
    358-1921               0  False            0.050000                  0             High      Loyalty Appreciation Package                      